In [2]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Testing groot to many predicates

In [3]:
predicates = {'target': ['actedin'],#, 'gene', 'author'],
              'source': ['accounttype']}#, 'tweets', 'accounttype']}

In [4]:
kb = json.loads(open('src/experiments/kb.txt').readline())

In [5]:
final_results = {}
source_pred = predicates['source']
target_pred = predicates['target']

for pred in zip(source_pred, target_pred):
    final_results[f'{pred[0]}->{pred[1]}'] = ''
    final_results[f'inf_res:{pred[0]}->{pred[1]}'] = ''
    final_results[f'groot_test:{pred[0]}->{pred[1]}'] = ''
    final_results[f'tree_test:{pred[0]}->{pred[1]}'] = ''

In [6]:
kb_source = kb['twitter']
kb_target = kb['yago2s']

In [7]:
pred_target = create_pred_target(kb_target)

In [8]:
for source, target in zip(source_pred, target_pred):
    print(f'Iniciando transferência {source}->{target}')

    twitter_dataset = datasets.load('twitter', kb_source, target=source, seed=441773) #facts, pos, neg
    yago2s_dataset = datasets.load('yago2s', kb_target, target=target, seed=441773) #facts, pos, neg
    
    target = f'recursion_{target}'
    
    train_facts_source, train_pos_source, train_neg_source = get_train_division(twitter_dataset)
    background_knowledge_src = boostsrl.modes(kb_source, [source], useStdLogicVariables=False, 
                                          maxTreeDepth=3, nodeSize=2, numOfClauses=8)
    model_src = boostsrl.train(background_knowledge_src, train_pos_source, train_neg_source, train_facts_source, trees=10)
    
    structured_src, new_src_struct = create_structured_trees(model_src)
    
    
    train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(yago2s_dataset)
    
    start = time.time()
    final_results[f'{source}->{target}'] = genetic(new_src_struct, target, source, train_pos, 
                                             train_neg, train_facts, kb_source, kb_target, pred_target,
                                              NUM_GEN=14, pop_size=5)
    print("FIM DO GROOT: ", time.time()-start)
    
    tr_train_pos = []
    tr_train_neg = []
    tr_train_facts = []

    for i in range(0, len(train_pos)):
        tr_train_pos.extend(train_pos[i])
        tr_train_neg.extend(train_neg[i])
        tr_train_facts.extend(train_facts[i])
        
    train = [tr_train_pos, tr_train_neg, tr_train_facts]
    test = [test_pos, test_neg, test_facts]
    
    individual = get_best_individual(final_results[f'{source}->{target}'][0].population)
    refine, transfer = get_refine_transfer(individual, source, target, 'twitter', 'yago2s')
    
    final_results[f'groot_test:{source}->{target}'], model = test_refine_transfer(kb_target, target, refine, transfer, train, test)
    
    final_results[f'inf_res:{source}->{target}'] = model.inference_results(target)
    
    final_results[f'tree_test:{source}->{target}'] = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)
    
    save_results(final_results, source, target, 'twitter', 'yago2s')
    
    print(f"Finalizando transferência {source}->{target} com tempo {time.time()-start} s")

Iniciando transferência accounttype->actedin


/usr/local/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


FileNotFoundError: [Errno 2] No such file or directory: 'boostsrl/train/models/WILLtheories/recursion_actedin_learnedWILLregressionTrees.txt'

In [9]:
!java -XX:+PrintFlagsFinal -version | grep -iE 'heapsize|permsize|threadstacksize'

Picked up _JAVA_OPTIONS: -Xmx4g
     intx CompilerThreadStackSize                  = 1024                                   {pd product} {default}
   size_t ErgoHeapSizeLimit                        = 0                                         {product} {default}
   size_t HeapSizePerGCThread                      = 43620760                                  {product} {default}
   size_t InitialHeapSize                          = 134217728                                 {product} {ergonomic}
   size_t LargePageHeapSizeThreshold               = 134217728                                 {product} {default}
   size_t MaxHeapSize                              = 4294967296                                {product} {command line}
   size_t MinHeapSize                              = 8388608                                   {product} {ergonomic}
    uintx NonNMethodCodeHeapSize                   = 5832780                                {pd product} {ergonomic}
    uintx NonProfiledCodeHeapSize    

In [9]:
!export _JAVA_OPTIONS="-Xmx4g"